# Table of Contents

1. **Library Imports**
    * Library Import

2. **Connect to Server and Create Database**
    * Define Server Connection Function
    * Create a New Database
    * Modify Server Connection Function, Create Database Connection Function
    * Define Query Execution Function

3. **Creating Tables**
    * Create Teacher Table
    * Create Remaining Tables
    * Define Foreign Key Relationships

4. **Populating Tables**
    * Populate Teacher Table
    * Populate Remaining Tables

5. **Reading Data**
    * Define Data Reading Function
    * Read Data from Database
    * Formatting Output into a List
    * Formatting Output into a List of Lists
    * Formatting Output into a pandas DataFrame

6. **Updating Records**
    * Updating Client Address

7. **Deleting Records**
    * Deleting a Course
    * Restoring the Course

8. **Creating Records from Lists**
    * Create Execute List Query Function
    * Add New Teachers

9. **Conclusion**
    * Conclusion


# 1. Import Libraries

Our first step will be to import the necessary libraries. We will use pyodbc and mariadb to connect to databases in LCL Logistics.

### Python Database Connector Libraries

1. **SQLite3** - Built into Python, used for accessing SQLite databases.
   - [SQLite3 Documentation](https://docs.python.org/3/library/sqlite3.html)

2. **MySQL Connector/Python** - A MySQL driver written in Python.
   - [MySQL Connector/Python Documentation](https://dev.mysql.com/doc/connector-python/en/)

3. **Psycopg2** - PostgreSQL adapter for Python.
   - [Psycopg2 Documentation](https://www.psycopg.org/docs/)

4. **SQLAlchemy** - SQL toolkit and Object-Relational Mapping (ORM) system.
   - [SQLAlchemy Documentation](https://www.sqlalchemy.org/)

5. **PyMongo** - MongoDB driver for Python.
   - [PyMongo Documentation](https://pymongo.readthedocs.io/en/stable/)

6. **PyODBC** - Open-source Python module for ODBC database connections.
   - [PyODBC Documentation](https://github.com/mkleehammer/pyodbc/wiki)

7. **cx_Oracle** - Python extension for Oracle database access.
   - [cx_Oracle Documentation](https://oracle.github.io/python-cx_Oracle/)


### Library imports for LCL DBs

In [1]:
import pyodbc
import mariadb
import os
from dotenv import load_dotenv

load_dotenv()

True

# 2. Connect to Server and Create Database
#### 2.1 - Define Server Connection Function
To establish a connection to a MariaDB server using Python for managing the logistics database, we utilize the `pyodbc` library. This function will attempt to connect to the server with the provided credentials and will output a success message if the connection is established successfully. Make sure to update the 'pw' variable with the actual password for your server.


Installing drivers on Mac

```bash
brew tap microsoft/mssql-release https://github.com/Microsoft/homebrew-mssql-release
brew update
brew install msodbcsql17 mssql-tools

In [9]:
def create_server_connection(server_name, database_name, user_name, user_password):
    connection = None
    try:
        connection = pyodbc.connect(
            'Driver={ODBC Driver 17 for SQL Server};'
            f'Server={server_name};'
            f'Database={database_name};'
            f'UID={user_name};'
            f'PWD={user_password}'
        )
        print("Database connection successful")
    except pyodbc.Error as err:
        print(f"Error: '{err}'")

    return connection

# Connection to MariaDB
connection = create_server_connection(os.environ["SERVER"], os.environ["DB"], os.environ["USER"], os.environ["PW"])

Error: '('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'ODBC Driver 17 for SQL Server' : file not found (0) (SQLDriverConnect)")'


In [3]:
from mariadb.connections import Connection


connection: Connection = mariadb.connect(
    user = os.environ["M_USER"],
    password = os.environ["M_PW"],
    host = os.environ["M_SERVER"],
    database = os.environ["M_DB"]
)

#### 2.2 - Create a New Database
To facilitate operations at LCL Logistics, this function will create a new database on the MariaDB server specifically for managing perishable logistics. We use the `cursor.execute()` method to run a SQL command that creates the database.


In [17]:

def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Exception as err:
        print(f"Error: '{err}'")

# SQL command to create a new database
create_database_query = "CREATE DATABASE oto"  # Name the database according to your logistics needs
create_database(connection, create_database_query)


Error: 'Access denied for user 'megatron'@'%' to database 'oto''


#### 2.3 - Define Query Execution Function
This function is essential for executing SQL queries directly on the MariaDB database managing logistics operations. It handles all operations, from data insertion to updates and deletions, crucial for managing perishable logistics efficiently.


In [4]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Exception as err:
        print(f"Error: '{err}'")

# Example SQL query to test the function
test_query = "CREATE TABLE IF NOT EXISTS inventory (id INT AUTO_INCREMENT PRIMARY KEY, item_name VARCHAR(255), quantity INT, expiry_date DATE)"
execute_query(connection, test_query)

Query successful


### 3. Creating Tables


#### 3.1 - Create Staff Table
To adapt this tutorial section for LCL Logistics, let's create a table that is directly relevant to logistics operations. We'll focus on a `staff` table that can be used to manage employee details including certifications and contact information.


In [5]:
create_staff_table = """
CREATE TABLE staff (
  staff_id INT PRIMARY KEY,
  first_name VARCHAR(40) NOT NULL,
  last_name VARCHAR(40) NOT NULL,
  role VARCHAR(20) NOT NULL,
  certification VARCHAR(20),
  dob DATE,
  tax_id INT UNIQUE,
  phone_no VARCHAR(20)
  );
"""

execute_query(connection, create_staff_table)

Query successful


#### 3.2 - Create Remaining Tables
For LCL Logistics, it's essential to manage data relevant to their core business functions. Therefore, let's create tables for `trucks`, `orders`, and `customers` which are more pertinent to logistics operations involving perishables.


In [6]:
# SQL command to create a customers table
create_customers_table = """
CREATE TABLE customers (
  customer_id INT PRIMARY KEY,
  customer_name VARCHAR(40) NOT NULL,
  address VARCHAR(60) NOT NULL,
  contact_phone VARCHAR(20)
);
"""

# SQL command to create a trucks table
create_trucks_table = """
CREATE TABLE trucks (
  truck_id INT PRIMARY KEY,
  license_plate VARCHAR(15) NOT NULL,
  model VARCHAR(40) NOT NULL,
  capacity_ton INT,
  maintenance_date DATE
);
"""

# SQL command to create an orders table
create_orders_table = """
CREATE TABLE orders (
  order_id INT PRIMARY KEY,
  product_name VARCHAR(40) NOT NULL,
  quantity INT NOT NULL,
  order_date DATE NOT NULL,
  delivery_date DATE,
  customer_id INT,
  truck_id INT,
  FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
  FOREIGN KEY (truck_id) REFERENCES trucks(truck_id)
);
"""

# Execute the queries to create the tables
execute_query(connection, create_customers_table)
execute_query(connection, create_trucks_table)
execute_query(connection, create_orders_table)

Query successful
Query successful
Query successful


### 3.3 - Define Foreign Key Relationships
To ensure data integrity within LCL Logistics' database, we will alter tables to create appropriate foreign key relationships. This adjustment focuses on tables relevant to logistics, specifically linking `orders` to `customers` and `trucks`, ensuring that order details relate directly to existing customers and trucks.


In [7]:
# SQL command to alter the orders table to add foreign key relationships
alter_orders_customer = """
ALTER TABLE orders
ADD FOREIGN KEY(customer_id)
REFERENCES customers(customer_id)
ON DELETE SET NULL;
"""

alter_orders_truck = """
ALTER TABLE orders
ADD FOREIGN KEY(truck_id)
REFERENCES trucks(truck_id)
ON DELETE SET NULL;
"""

# SQL command to create a final table for managing deliveries
create_delivery_table = """
CREATE TABLE delivery (
  order_id INT,
  truck_id INT,
  delivery_status VARCHAR(20),
  delivery_date DATE,
  PRIMARY KEY(order_id, truck_id),
  FOREIGN KEY(order_id) REFERENCES orders(order_id) ON DELETE CASCADE,
  FOREIGN KEY(truck_id) REFERENCES trucks(truck_id) ON DELETE CASCADE
);
"""

# Execute the SQL commands to alter tables and create new foreign key relationships
execute_query(connection, alter_orders_customer)
execute_query(connection, alter_orders_truck)
execute_query(connection, create_delivery_table)

Query successful
Query successful
Query successful


### 4. Populate Tables


#### 4.1 - Populate Staff Table
As part of the data management strategy for LCL Logistics, let's populate the `staff` table. This action involves inserting relevant data for staff members that might be involved in logistics operations, such as drivers, warehouse managers, etc.


In [10]:

# SQL command to insert data into the staff table
pop_staff = """
INSERT INTO staff VALUES
(1, 'James', 'Smith', 'Driver', 'Class A', '1985-04-20', 12345, '+491774553676'),
(2, 'Stefanie', 'Martin', 'Dispatcher', NULL, '1970-02-17', 23456, '+491234567890'), 
(3, 'Steve', 'Wang', 'Logistics Manager', 'Project Management', '1990-11-12', 34567, '+447840921333'),
(4, 'Friederike', 'Müller-Rossi', 'Fleet Coordinator', 'Mechanical Cert', '1987-07-07', 45678, '+492345678901'),
(5, 'Isobel', 'Ivanova', 'Inventory Specialist', 'Supply Chain Mx', '1963-05-30', 56789, '+491772635467'),
(6, 'Niamh', 'Murphy', 'HR Manager', 'HR Training', '1995-09-08', 67890, '+491231231232');
"""

# Execute the query to populate the staff table
execute_query(connection, pop_staff)


Query successful


#### 4.2 - Populate Remaining Tables
We'll now insert sample data into the `customers`, `trucks`, and `orders` tables, which are central to the operations at LCL Logistics. This step is crucial for setting up a realistic dataset to test and manage logistics operations effectively.


In [11]:

# SQL command to insert data into the customers table
pop_customers = """
INSERT INTO customers VALUES
(1, 'BigBox Retailers', '123 Commerce Rd.', '+18005551234'),
(2, 'Tech Gadgets Ltd.', '456 Innovation Dr.', '+18007772345'),
(3, 'Farm Fresh Produce', '789 Country Rd.', '+18003334455');
"""

# SQL command to insert data into the trucks table
pop_trucks = """
INSERT INTO trucks VALUES
(101, 'AB123CD', 'Volvo FMX', 20, '2023-01-10'),
(102, 'EF456GH', 'Mercedes Actros', 15, '2023-02-20'),
(103, 'IJ789KL', 'Scania S-series', 25, '2023-03-15');
"""

# SQL command to insert data into the orders table
pop_orders = """
INSERT INTO orders VALUES
(1001, 'Apples', 1200, '2024-03-01', '2024-03-03', 3, 101),
(1002, 'Laptops', 50, '2024-03-02', '2024-03-04', 2, 102),
(1003, 'Tomatoes', 800, '2024-03-02', '2024-03-03', 3, 103);
"""

# Execute the queries to populate the tables
execute_query(connection, pop_customers)
execute_query(connection, pop_trucks)
execute_query(connection, pop_orders)


Query successful
Query successful
Query successful


### 5. Reading Data


#### 5.1 - Define Data Reading Function
To access and analyze the data within the logistics database effectively, we need a function capable of executing read queries and fetching the results. This function will be integral to reviewing data related to trucks, orders, and customer interactions, crucial for operational decisions at LCL Logistics.


In [12]:
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        for row in result:
            print(row)
    except pyodbc.Error as err:
        print(f"Error: '{err}'")
    return result

# Example usage of the read_query function
# This is an example to fetch all entries from the customers table
query = "SELECT * FROM customers;"
results = read_query(connection, query)


(1, 'BigBox Retailers', '123 Commerce Rd.', '+18005551234')
(2, 'Tech Gadgets Ltd.', '456 Innovation Dr.', '+18007772345')
(3, 'Farm Fresh Produce', '789 Country Rd.', '+18003334455')


### 5.2 - Read Data from Database
Let's try this with a simple query to begin with.


In [13]:

q1 = """
SELECT *
FROM orders;
"""

results = read_query(connection, q1)

for result in results:
  print(result)

(1001, 'Apples', 1200, datetime.date(2024, 3, 1), datetime.date(2024, 3, 3), 3, 101)
(1002, 'Laptops', 50, datetime.date(2024, 3, 2), datetime.date(2024, 3, 4), 2, 102)
(1003, 'Tomatoes', 800, datetime.date(2024, 3, 2), datetime.date(2024, 3, 3), 3, 103)
(1001, 'Apples', 1200, datetime.date(2024, 3, 1), datetime.date(2024, 3, 3), 3, 101)
(1002, 'Laptops', 50, datetime.date(2024, 3, 2), datetime.date(2024, 3, 4), 2, 102)
(1003, 'Tomatoes', 800, datetime.date(2024, 3, 2), datetime.date(2024, 3, 3), 3, 103)


#### Additional queries for the provided schemas


In [14]:
# Query to select customer last names and dates of birth
q2 = """
SELECT customer_name, contact_phone
FROM customers;
"""

# Query to select all truck records ordered by maintenance date descending where the capacity is greater than 5 tons
q3 = """
SELECT *
FROM trucks
WHERE capacity_ton > 5
ORDER BY maintenance_date DESC;
"""

# Query to select customer names and contact phones for customers who have placed orders
q4 = """
SELECT customer_name, contact_phone
FROM customers
WHERE customer_id IN (SELECT DISTINCT customer_id FROM orders);
"""

# Query to select order details along with customer and truck information for orders delivered outside the school
q5 = """
SELECT orders.order_id, orders.product_name, orders.quantity, orders.order_date, orders.delivery_date, 
       customers.customer_name, customers.address AS customer_address, trucks.license_plate, trucks.model
FROM orders
JOIN customers ON orders.customer_id = customers.customer_id
JOIN trucks ON orders.truck_id = trucks.truck_id
WHERE orders.delivery_date IS NOT NULL;
"""

results = read_query(connection, q5)

# Display results
for result in results:
    print(result)


(1002, 'Laptops', 50, datetime.date(2024, 3, 2), datetime.date(2024, 3, 4), 'Tech Gadgets Ltd.', '456 Innovation Dr.', 'EF456GH', 'Mercedes Actros')
(1001, 'Apples', 1200, datetime.date(2024, 3, 1), datetime.date(2024, 3, 3), 'Farm Fresh Produce', '789 Country Rd.', 'AB123CD', 'Volvo FMX')
(1003, 'Tomatoes', 800, datetime.date(2024, 3, 2), datetime.date(2024, 3, 3), 'Farm Fresh Produce', '789 Country Rd.', 'IJ789KL', 'Scania S-series')
(1002, 'Laptops', 50, datetime.date(2024, 3, 2), datetime.date(2024, 3, 4), 'Tech Gadgets Ltd.', '456 Innovation Dr.', 'EF456GH', 'Mercedes Actros')
(1001, 'Apples', 1200, datetime.date(2024, 3, 1), datetime.date(2024, 3, 3), 'Farm Fresh Produce', '789 Country Rd.', 'AB123CD', 'Volvo FMX')
(1003, 'Tomatoes', 800, datetime.date(2024, 3, 2), datetime.date(2024, 3, 3), 'Farm Fresh Produce', '789 Country Rd.', 'IJ789KL', 'Scania S-series')


### 5.3 - Formatting Output into a List
Now we can assign the results to a list, to use further in our python applications or scripts.

The following code returns the results of our query as a list of tuples.


In [15]:
#Initialise empty list
from_db = []

# Loop over the results and append them into our list, different styles

# Returns a list of tuples
for result in results:
  result = result
  from_db.append(result)
    
print(from_db)

[(1002, 'Laptops', 50, datetime.date(2024, 3, 2), datetime.date(2024, 3, 4), 'Tech Gadgets Ltd.', '456 Innovation Dr.', 'EF456GH', 'Mercedes Actros'), (1001, 'Apples', 1200, datetime.date(2024, 3, 1), datetime.date(2024, 3, 3), 'Farm Fresh Produce', '789 Country Rd.', 'AB123CD', 'Volvo FMX'), (1003, 'Tomatoes', 800, datetime.date(2024, 3, 2), datetime.date(2024, 3, 3), 'Farm Fresh Produce', '789 Country Rd.', 'IJ789KL', 'Scania S-series')]


### 5.4 - Formatting Output into a List of Lists
If we want to, we can make this return a list of lists instead, like so:


In [16]:
# Returns a list of lists
from_db = []

for result in results:
  result = list(result)
  from_db.append(result)
    
print(from_db)

[[1002, 'Laptops', 50, datetime.date(2024, 3, 2), datetime.date(2024, 3, 4), 'Tech Gadgets Ltd.', '456 Innovation Dr.', 'EF456GH', 'Mercedes Actros'], [1001, 'Apples', 1200, datetime.date(2024, 3, 1), datetime.date(2024, 3, 3), 'Farm Fresh Produce', '789 Country Rd.', 'AB123CD', 'Volvo FMX'], [1003, 'Tomatoes', 800, datetime.date(2024, 3, 2), datetime.date(2024, 3, 3), 'Farm Fresh Produce', '789 Country Rd.', 'IJ789KL', 'Scania S-series']]


In [ ]:
# Returns a list of lists and then creates a pandas DataFrame
from_db = []

for result in results:
  result = list(result)
  from_db.append(result)


columns = ["course_id", "course_name", "language", "client_name", "address"]
df = pd.DataFrame(from_db, columns=columns)

display(df)

In [17]:
# Imports pandas library
import pandas as pd

# List to store results from the database
from_db = []

# Appending results to the list
for result in results:
    result = list(result)
    from_db.append(result)

# Define column names for DataFrame
columns = ["order_id", "product_name", "quantity", "order_date", "delivery_date", "customer_name", "customer_address", "license_plate", "model"]

# Create DataFrame using pandas
df = pd.DataFrame(from_db, columns=columns)

# Display the DataFrame
display(df)


,order_id,product_name,quantity,order_date,delivery_date,customer_name,customer_address,license_plate,model
0,1002,Laptops,50,2024-03-02,2024-03-04,Tech Gadgets Ltd.,456 Innovation Dr.,EF456GH,Mercedes Actros
1,1001,Apples,1200,2024-03-01,2024-03-03,Farm Fresh Produce,789 Country Rd.,AB123CD,Volvo FMX
2,1003,Tomatoes,800,2024-03-02,2024-03-03,Farm Fresh Produce,789 Country Rd.,IJ789KL,Scania S-series


## 6. Updating Records
Sometimes we will need to update our Database. We can do this very easily using our execute_query function alongside the SQL UPDATE statement.



### 6.1 - Updating Client Address
The School receives notification that the Big Business Federation has moved office, and now they are located at 23 Fingiertweg, 14534 Berlin. We can change that in our database like so:


In [18]:
# SQL UPDATE statement to update customer's address
update = """
UPDATE customers 
SET address = '23 Fingiertweg, 14534 Berlin' 
WHERE customer_id = <customer_id_to_update>;
"""

# Execute the update query
execute_query(connection, update)


Error: 'You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near '<customer_id_to_update>' at line 3'


Let's see if that worked.

In [ ]:
q1 = """
SELECT *
FROM customer
WHERE customer_id = <customer_id_to_update>;
"""

results = read_query(connection, q1)

for result in results:
  print(result)

### 7. Deleting Records



#### 7.1 - Deleting a Course

We can also use our `execute_query` function to delete records, by using `DELETE FROM`.

Let's try this with our `orders` table. First, let's remind ourselves of the courses contained in the table.


In [21]:
q1 = """
SELECT *
FROM orders;
"""

results = read_query(connection, q1)

from_db = []

for result in results:
  print(result)

(1002, 'Laptops', 50, datetime.date(2024, 3, 2), datetime.date(2024, 3, 4), 2, 102)
(1003, 'Tomatoes', 800, datetime.date(2024, 3, 2), datetime.date(2024, 3, 3), 3, 103)
(1002, 'Laptops', 50, datetime.date(2024, 3, 2), datetime.date(2024, 3, 4), 2, 102)
(1003, 'Tomatoes', 800, datetime.date(2024, 3, 2), datetime.date(2024, 3, 3), 3, 103)


Let's delete the order with order_id 1001  - 'Apples'. For this we will use the DELETE SQL command.

In [20]:
delete_course = """
DELETE FROM orders WHERE order_id = 1001;
"""

execute_query(connection, delete_course)

Query successful


## 8. Creating Records from Lists


We saw in Section 4 that we can use the SQL INSERT command in our `execute_query` function to insert records into our tables.

MySQL Connector also gives us a way to do this in a more 'pythonic' fashion, using a list of tuples as our input, where each tuple contains the data we wish to insert into our table. This is extremely useful for updating our database with data which may have been generated by an application we have written in Python, such as logs of user activity on a social media app, for example.

To do this, we will use the `executemany()` method, instead of the simpler `execute()` method we have been using thus far.

This method is also more secure if our database is open to our users at any point, as it helps to prevent against SQL Injection attacks, which can damage or even destroy our whole database.


### 8.1 - Create Execute List Query Function

To see how we can do this, let's add a couple of new teachers to our teacher table.

First, let's modify our `execute_query` function to use `executemany()` and to accept one more argument.


In [22]:
def execute_list_query(connection, sql, val):
    cursor = connection.cursor()
    try:
        cursor.executemany(sql, val)
        connection.commit()
        print("Query successful")
    except Exception as err:
        print(f"Error: '{err}'")

### 8.2 - Add New Staff

Now let's create a list containing the data for our new staff (each stored within a tuple), and the SQL command to perform our action.

Notice that the SQL command requires a '%s' placeholder for each of the columns we wish to act upon, so in this case we need 8 for the 8 columns we wish to add values for.


In [23]:
# SQL INSERT statement to insert data into the staff table
sql = '''
    INSERT INTO staff (staff_id, first_name, last_name, role, certification, dob, tax_id, phone_no) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    '''

# Values to be inserted into the staff table
val = [
    (7, 'Hank', 'Dodson', 'ENG', None, '1991-12-23', 11111, '+491772345678'), 
    (8, 'Sue', 'Perkins', 'MAN', 'ENG', '1976-02-02', 22222, '+491443456432')
]

# Execute the insert query
execute_list_query(connection, sql, val)


Query successful


## 9. Conclusion

### 9.1 - Conclusion

We have covered a lot of ground in this tutorial. From using Python and MySQL Connector to create an entirely new database in MySQL Server, creating tables, defining their relationships to one another and populating them with data. We have covered how to Create, Read, Update and Delete data in our database.

We have looked at how to extract data from existing databases and load them into pandas DataFrames, ready for analysis and further work taking advantage of all the possibilities offered by the PyData stack. Going the other direction, we have also learned how to take data generated by our Python scripts and applications, and write those into a database where they can be safely stored for later retrieval.

I hope it is clear just how powerful each of these programming languages can be for Data Analysts. Using them together makes them even stronger.
